# Checkers
I have decided to join the Board class and the Game class into one class. The way they have implemented the Board class makes it difficult to implement a checkers board. At the moment it just stores {coordinate, player}. In checkers we need to move players, delete players, promote players etc.. Of course we could modify it, but from an OOP point of view I think it is fine for the Checkers game to possess and control a board internally.

We start with two lists of pieces. The boolean next to the location (i,j) is the king status. If the piece is taken we replace its location with None. That is, the lists P[0] and P[1] will always have size 12.

In [35]:
# A standard game of checkers. 8x8 board.
class Checkers():
    def __init__(self):
        # initial positions
        W = [(x,y) for y in range(0,3) for x in range(y%2,8,2)]
        B = [(x,y) for y in range(5,8) for x in range(y%2,8,2)]
        self.P = {'W' : W,'B' : B}
        # we record king status as a mapping from location to boolean
        W_King = {location : False for location in W}
        B_King = {location : False for location in B}
        self.Kings = {'W' : W_King,'B' : B_King}
        self.to_move = 'W'

    def find_moves(self, piece):
        # There are 4 possible moves.
        turn = self.to_move
        if(turn == 'W'):
            moves = []
            if(piece[1] < 7):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] + 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] + 1))
            if(self.Kings[turn][piece] and piece[1] > 0):
                if(piece[0] > 0):
                    moves.append((piece[0] - 1, piece[1] - 1))
                if(piece[0] < 7):
                    moves.append((piece[0] + 1, piece[1] - 1))
            available_moves = []
            for m in moves:
                if m in self.P['W']:
                    continue
                available_moves.append(m)
            return available_moves
            
        if(turn == 'B'):
            moves = []
            return moves
            
    def actions(self):
        for i in range(12):
            turn = self.to_move
            # If the piece is taken, just continue.
            if self.P[turn][i] == None: continue
            moves = self.find_moves(self.P[turn][i])
            print(self.P[turn][i], moves)
            
            

In [36]:
c = Checkers()
c.actions()

(0, 0) []
(2, 0) []
(4, 0) []
(6, 0) []
(1, 1) []
(3, 1) []
(5, 1) []
(7, 1) []
(0, 2) [(1, 3)]
(2, 2) [(1, 3), (3, 3)]
(4, 2) [(3, 3), (5, 3)]
(6, 2) [(5, 3), (7, 3)]
